In [1]:
# p288 グローアルゴリズムの実装
import numpy as np

def span_includes(A, v, debug=False):
    rank_a  = np.linalg.matrix_rank(np.array(A).transpose())
    rank_ab = np.linalg.matrix_rank(np.array(A + [v]).transpose())
    
    if debug:
        print(rank_a, rank_ab)
    
    # 唯一解かどうかは調べなくていいだろうか？
    return rank_a == rank_ab

def grow(T):
    B = []
    for v in T:
        if not span_includes(B, v):
            B.append(v)
    return B

T = [
    [1, 0, 0],
    [0, 1, 0],
    [2, 4, 0],
    [0, 0, 1],
    [1, 1, 0]
]

grow(T)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [2]:
grow(reversed(T))

[[1, 1, 0], [0, 0, 1], [2, 4, 0]]

In [3]:
# p288 シュリンクアルゴリズムの実装
from copy import copy

def shrink(T):
    Tc = np.array(T)
    live = [True] * len(T)
    for i, v in enumerate(T):
        live[i] = False
        if not span_includes(Tc[live].tolist(), v):
            live[i] = True
    return [v for i, v in enumerate(T) if live[i]]

shrink(T)

[[2, 4, 0], [0, 0, 1], [1, 1, 0]]

In [4]:
grow(shrink(T))

[[2, 4, 0], [0, 0, 1], [1, 1, 0]]

In [5]:
shrink(grow(T))

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [6]:
C = [
    [2, 0, 0],
    [0, 2, 0],
    [1, 1, 2]
]
grow(C)

[[2, 0, 0], [0, 2, 0], [1, 1, 2]]

In [21]:
# 5.9 遠近法によるレンダリング
from GF2 import one
from math import sqrt, pi
from matutil import coldict2mat, mat2coldict
from solver import solve
from vec import Vec
from vecutil import list2vec
from plotting import plot
from operator import itemgetter

def vec2rep(veclist, v):
    A = coldict2mat({i: col for i, col in enumerate(veclist)})
    return solve(A, v)

In [8]:
L = [
    [0,0,0], [1,0,0], [0,1,0], [1,1,0],
    [0,0,1], [1,0,1], [0,1,1], [1,1,1]
]
corners = [list2vec(v) for v in L]

In [9]:
def line_segment(pt1, pt2, samples=200):
    return [((i/samples)*pt1 + ((1-i/samples)*pt2)) for i in range(samples+1)]

line_segments = [
    line_segment(corners[i], corners[j]) for i, j in
    [(0,1), (2,3), (0,2), (1,3), (4,5), (6,7), (4,6), (5,7), (0,4), (1,5), (2,6), (3,7)]
]

pts = sum(line_segments, [])
len(pts)

2412

In [10]:
def pixel(x):
    return (x[0], x[1])

def scale_down(x):
    return list2vec([x[0]/x[2], x[1]/x[2], 1])

In [11]:
shifted_pts = [v + list2vec([1, 1, 8]) for v in pts]

cb = [
    list2vec([1/100, 0, 0]),
    list2vec([0, 1/100, 0]),
    list2vec([0, 0, 1])
]

reps = [vec2rep(cb, v) for v in shifted_pts]
in_camera_plane = [scale_down(u) for u in reps]
pixels = [pixel(u) for u in in_camera_plane]

plot(pixels, 30, 1)

max(map(itemgetter(0), pixels))

open /mnt/d/linux/study/coding-the-matrix/notebook/tmp/tmpapjebn2t.html in browser


25.0

In [12]:
shifted_pts = [v + list2vec([-0.5, -0.5, 8]) for v in pts]

cb = [
    list2vec([1/100, 0, 0]),
    list2vec([0, 1/100, 0]),
    list2vec([0, 0, 1])
]

reps = [vec2rep(cb, v) for v in shifted_pts]
in_camera_plane = [scale_down(u) for u in reps]
pixels = [pixel(u) for u in in_camera_plane]

plot(pixels, 30, 1)

max(map(itemgetter(0), pixels))

open /mnt/d/linux/study/coding-the-matrix/notebook/tmp/tmph_gwm3vg.html in browser


6.25

In [13]:
span_includes([[1,-1,1],[1,2,-2]], [0,0,0], debug=True)

2 2


True

In [26]:
# 透視補正
from vec import Vec
from mat import Mat
from matutil import rowdict2mat
from solver import solve

def move2board(y):
    return Vec({'y1', 'y2', 'y3'}, {'y1': y['y1'] / y['y3'], 'y2': y['y2'] / y['y3'], 'y3': 1})

D = {(r, c) for r in {'y1', 'y2', 'y3'} for c in {'x1', 'x2', 'x3'}}

def make_equations(x1, x2, w1, w2):
    u = Vec(D, {('y3', 'x1'): w1 * x1, ('y3', 'x2'): w1 * x2, ('y3', 'x3'): w1 * 1, ('y1', 'x1'): -x1, ('y1', 'x2'): -x2, ('y1', 'x3'): -1})
    v = Vec(D, {('y3', 'x1'): w2 * x1, ('y3', 'x2'): w2 * x2, ('y3', 'x3'): w2 * 1, ('y2', 'x1'): -x1, ('y2', 'x2'): -x2, ('y2', 'x3'): -1})
    return [u, v]

b = Vec(set(range(9)), {8: 1})

def make_nine_equations(corners):
    vecs = []
    # for x, w in zip(corners, [(0, 0), (0, 1), (1, 0), (1, 1)]):
    for x, w in zip(corners, [(0, 0), (0, 1), (5, 0), (5, 1)]):
        vecs.extend(make_equations(x[0], x[1], w[0], w[1]))
    vecs.append(Vec(D, {('y1', 'x1'): 1}))
    return vecs

veclist = make_nine_equations([(358, 36), (329, 597), (592, 157), (580, 483)])

L = rowdict2mat({i: v for i, v in enumerate(veclist)})

hvec = solve(L, b)

print(hvec)


 ('y1', 'x1') ('y1', 'x2') ('y1', 'x3') ('y2', 'x1') ('y2', 'x2') ('y2', 'x3') ('y3', 'x1') ('y3', 'x2') ('y3', 'x3')
---------------------------------------------------------------------------------------------------------------------
            1       0.0517         -360      -0.0763        0.148           22       -0.144     -0.00234          134


In [20]:
from image_mat_util import file2mat, mat2display

(X_pts, colors) = file2mat('image/board.png', ('x1', 'x2', 'x3'))

In [16]:
H = Mat(({'y1', 'y2', 'y3'}, {'x1', 'x2', 'x3'}), hvec.f)
Y_pts = H * X_pts

In [18]:
def mat_move2board(Y):
    return coldict2mat({c: move2board(v) for c, v in mat2coldict(Y).items()})

Y_board = mat_move2board(Y_pts)

In [22]:
mat2display(Y_board, colors, ('y1', 'y2', 'y3'), scale=100, xmin=None, ymin=None)

open /mnt/d/linux/study/coding-the-matrix/notebook/tmp/tmpjwtr_1wz.html in browser
Hit Enter once the image is displayed.... 


In [27]:
# p313 課題 5.12.9
# cit.png

veclist = make_nine_equations([(135, 62), (135, 100), (10, 85), (8, 112)])
L = rowdict2mat({i: v for i, v in enumerate(veclist)})
hvec = solve(L, b)

print(hvec)


 ('y1', 'x1') ('y1', 'x2') ('y1', 'x3') ('y2', 'x1') ('y2', 'x2') ('y2', 'x3') ('y3', 'x1') ('y3', 'x2') ('y3', 'x3')
---------------------------------------------------------------------------------------------------------------------
            1     1.73E-16         -135       -0.175       -0.954         82.8      -0.0874      -0.0213        -22.3


In [28]:
(X_pts, colors) = file2mat('image/cit.png', ('x1', 'x2', 'x3'))
H = Mat(({'y1', 'y2', 'y3'}, {'x1', 'x2', 'x3'}), hvec.f)
Y_pts = H * X_pts
Y_board = mat_move2board(Y_pts)

mat2display(Y_board, colors, ('y1', 'y2', 'y3'), scale=100, xmin=None, ymin=None)

open /mnt/d/linux/study/coding-the-matrix/notebook/tmp/tmpmdsx5sl1.html in browser
Hit Enter once the image is displayed.... 
